生成数据集

In [21]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [22]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
'''
根据真实的w，和b生成人造数据集
features这里是指X（各种影响结果y的特征），labels是指结果Y，1000是样本数量。
synthetic_data是一个d2l中的人造数据的函数。
'''

读取数据集

构造一个数据迭代器，每次读取指定数量个小批量随机样本。
通过DataLoader(指定数据集中的一部分数据)指定batch_size（数据大小）。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。


In [23]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [37]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)
"""这里我们使用iter构造Python迭代器，并使用next从迭代器中获取第一项"""

'这里我们使用iter构造Python迭代器，并使用next从迭代器中获取第一项'

In [39]:
next(iter(data_iter))

[tensor([[-0.8618, -0.3128],
         [ 0.5181,  0.5058],
         [-0.6576, -0.2590],
         [-0.9924,  0.4049],
         [ 1.3200, -0.2415],
         [ 0.5535, -0.6537],
         [-0.2792, -0.3875],
         [ 1.1205,  0.6257],
         [ 1.2821, -0.8802],
         [-1.0154,  0.0015]]),
 tensor([[3.5442],
         [3.5216],
         [3.7693],
         [0.8360],
         [7.6516],
         [7.5316],
         [4.9735],
         [4.3209],
         [9.7550],
         [2.1518]])]

定义模型

对于标准深度学习模型，我们可以[使用框架的预定义好的层]。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 
我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 
在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。
回顾 :对于单层网络架构,这一单层被称为全连接层（fully-connected layer）， 因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。
在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [26]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

(初始化模型参数)

在使用net之前，我们需要初始化模型参数。 如在线性回归模型中的权重和偏置。 深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。


In [ ]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

定义损失函数

[计算均方误差使用的是MSELoss类，也称为平方𝐿2范数]。 默认情况下，它返回所有样本损失的平均值。

In [27]:
loss = nn.MSELoss()

定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们(实例化一个SGD实例)时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值（学习率），这里设置为0.03。


In [28]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练

使用高级API我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。 当我们需要更复杂的模型时，高级API的优势将大大增加。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:

    通过调用net(X)生成预测并计算损失l（前向传播）。
    通过进行反向传播来计算梯度。
    通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。


In [31]:
num_epochs = 3 #迭代周期数
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad() #梯度清零
        l.backward()
        trainer.step() #更新模型
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000104
epoch 2, loss 0.000104
epoch 3, loss 0.000104


下面我们[比较生成数据集的真实参数和通过有限数据训练获得的模型参数]。 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。 正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [32]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0003, 0.0003])
b的估计误差： tensor([3.3855e-05])


小结

    我们可以使用PyTorch的高级API更简洁地实现模型。
    在PyTorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。
    我们可以通过_结尾的方法将参数替换，从而初始化参数。